In [1]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column,row
from bokeh.models import Div
from bokeh.palettes import Spectral6

import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

output_notebook();

Loading BokehJS ...

# chifflot-8.lille.grid5000.fr

## Data sources

In [ ]:
data_chifflot8_cpu = json.load(open('chifflot-8.lille.grid5000.fr/results_cpu.json', 'r'))
labels_chifflot8_cpu = [key for key, value in data_chifflot8_cpu.items()]

data_chifflot8_gpu = json.load(open('chifflot-8.lille.grid5000.fr/results_gpu.json', 'r'))
labels_chifflot8_gpu = [key for key, value in data_chifflot8_gpu.items()]

data_chifflot8_sycl_cpu = json.load(open('chifflot-8.lille.grid5000.fr/results_sycl_cpu.json', 'r'))
labels_chifflot8_sycl_cpu = [key for key, value in data_chifflot8_sycl_cpu.items()]

data_chifflot8_sycl_gpu = json.load(open('chifflot-8.lille.grid5000.fr/results_sycl_gpu.json', 'r'))
labels_chifflot8_sycl_gpu = [key for key, value in data_chifflot8_sycl_gpu.items()]

## Performace CPU

In [ ]:
# Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU
colors = ['green', 'red', 'blue', 'darkturquoise', 'coral']
i = 0

p1 = figure(title="Time", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'
p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

errors_total = list()

for label in labels_chifflot8_cpu:
    times = list()
    errors = list()
    data_err = data_chifflot8_cpu[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))
    
    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    
    errors_total += [errors]
    errors = list(map(float, errors))
    errors_t = dict()
    errors_t['sizes'] = sizes
    for key, value in zip(labels_chifflot8_cpu, errors_total):
        errors_t[key] = value
        
    source_chifflot8_cpu_errors = ColumnDataSource(pd.DataFrame(errors_t))
    
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending=True)
    data_plot = data_plot.reset_index(drop=True)
    
    source_chifflot8_cpu = ColumnDataSource(data_plot)
    
    p1.square(source=source_chifflot8_cpu, x='size', y='time', legend_label=labels_chifflot8_cpu[i], fill_color=None, line_color=colors[i])
    p1.line(source=source_chifflot8_cpu, x='size', y='time', line_color=colors[i])
    
    if label != 'serial':
        #p2.line(source=source_cpu_errors, x='sizes', y=label, legend_label=labels[i], color=colors[i])
        p2.square(source=source_chifflot8_cpu_errors, x='sizes', y=label, legend_label=labels_chifflot8_cpu[i], color=colors[i])
    
    if label == 'mkl':
        data_mkl_chifflot8 = data_plot.copy(deep=True)
        error_mkl_chifflot8 = pd.DataFrame(errors_t)
        
    if label == 'openMP':
        data_openMP_chifflot8 = data_plot.copy(deep=True)
        error_openMP_chifflot8 = pd.DataFrame(errors_t)
        
    i=i+1

maxi = max(max([sublist[:] for sublist in errors_total]))

p2.y_range.end = maxi*1.1
p1.legend.location = "top_left"
p2.legend.location = "top_left"

#show(gridplot([p1, p2], ncols=2))
show(column(Div(text=" Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU"), row(p1, p2)))

In [ ]:
# Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU
p1 = figure(title="Execution", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

source_mkl_chifflot8 = ColumnDataSource(data_mkl_chifflot8)
source_mkl_errors_chifflot8 = ColumnDataSource(error_mkl_chifflot8)
source_openmp_chifflot8 = ColumnDataSource(data_openMP_chifflot8)
source_openmp_errors_chifflot8 = ColumnDataSource(error_openMP_chifflot8)

p1.square(source=source_mkl_chifflot8, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
p1.line(source=source_mkl_chifflot8, x='size', y='time', line_color='blue')
p1.square(source=source_openmp_chifflot8, x='size', y='time', legend_label='openMP', fill_color=None, line_color='darkturquoise')
p1.line(source=source_openmp_chifflot8, x='size', y='time', line_color='darkturquoise')

p1.legend.location = "top_left"

p2.square(source=source_mkl_errors_chifflot8, x='sizes', y='mkl', legend_label='mkl', color='blue', fill_alpha=0.4)
#p2.line(source=source_mkl_errors, x='sizes', y='mkl', line_color='blue')
p2.square(source=source_openmp_errors_chifflot8, x='sizes', y='openMP', legend_label='openMP', color='darkturquoise', fill_alpha=0.4)
#p2.line(source=source_openmp_errors, x='sizes', y='openMP', line_color='darkturquoise')
p2.y_range.end = maxi*1.1
p2.legend.location = "top_left"

#show(gridplot([p1, p2], ncols=2))
show(column(Div(text="Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU"), row(p1, p2)))

In [ ]:
# SyCL
i = 0

colors = ['green', 'red', 'blue', 'black', 'coral']
p1 = figure(title="Time", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'


for label in labels_chifflot8_sycl_cpu:
    times = list()
    errors = list()
    data_err = data_chifflot8_sycl_cpu[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors = list(map(float, errors))
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    
    source_sycl_chifflot8 = ColumnDataSource(data_plot)
    
    p1.square(source=source_sycl_chifflot8, x='size', y='time', legend_label=labels_chifflot8_sycl_cpu[i], fill_color=None, line_color='darkturquoise')
    p1.line(source=source_sycl_chifflot8, x='size', y='time', line_color='darkturquoise')
    
    p1.square(source=source_mkl_chifflot8, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
    p1.line(source=source_mkl_chifflot8, x='size', y='time', line_color='blue')
    
    i=i+1


show(column(Div(text="Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU"), row(p1)))

## Performace GPU

In [ ]:
#Nvidia Tesla V100-PCIE-32GB (32 GiB)
i = 0

colors = ['mediumseagreen', 'tomato', 'dodgerblue', 'deeppink', 'darkviolet']

p1 = figure(title="Time", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

errors_total = list()

for label in labels_chifflot8_gpu:
    times = list()
    errors = list()
    data_err = data_chifflot8_gpu[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors_total += [errors]
    errors = list(map(float, errors))
    errors_t = dict()
    errors_t['sizes'] = sizes
    for key, value in zip(labels_chifflot8_gpu, errors_total):
        errors_t[key] = value
        
    source_chifflot8_gpu_errors = ColumnDataSource(pd.DataFrame(errors_t))
    
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    
    source_chifflot8_gpu = ColumnDataSource(data_plot)
    
    p1.square(source=source_chifflot8_gpu, x='size', y='time', legend_label=labels_chifflot8_gpu[i], fill_color=None, line_color=colors[i])
    p1.line(source=source_chifflot8_gpu, x='size', y='time', line_color=colors[i])
    
    if label != 'serial':
        p2.square(source=source_chifflot8_gpu_errors, x='sizes', y=label, legend_label=labels_chifflot8_gpu[i], color=colors[i])
        #p2.line(source=source_gpu_errors, x='sizes', y=label, legend_label=labels[i], color=colors[i])
    
    if label == 'cuBLAS_Tensor':
        data_cublas_tensor_chifflot8 = data_plot.copy(deep = True)
        error_cublas_tensor_chifflot8 = pd.DataFrame(errors_t)
    if label == 'cuBLAS':
        data_cublas_chifflot8 = data_plot.copy(deep = True)
        error_cublas_chifflot8 = pd.DataFrame(errors_t)
        
    i=i+1

maxi = max(max([sublist[:] for sublist in errors_total]))


p2.y_range.end = maxi*1.1

p1.legend.location = "top_left"
p2.legend.location = "top_left"

#show(gridplot([p1, p2], ncols=2))
show(column(Div(text="Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU - Nvidia Tesla V100-PCIE-32GB (32 GiB)"), row(p1, p2)))

In [ ]:
p1 = figure(title="Times", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'


colors = ['green', 'red', 'blue', 'black', 'coral']

source_cublasT_chifflot8 = ColumnDataSource(data_cublas_tensor_chifflot8)
source_cublasT_errors_chifflot8 = ColumnDataSource(error_cublas_tensor_chifflot8)
source_cublas_chifflot8 = ColumnDataSource(data_cublas_chifflot8)
source_cublas_errors_chifflot8 = ColumnDataSource(error_cublas_chifflot8)

p1.square(source=source_cublasT_chifflot8, x='size', y='time', legend_label='cuBLAS_Tensor', fill_color=None, line_color='tomato')
p1.line(source=source_cublasT_chifflot8, x='size', y='time', line_color='tomato')
p1.square(source=source_cublas_chifflot8, x='size', y='time', legend_label='cuBLAS', fill_color=None, line_color='mediumseagreen')
p1.line(source=source_cublas_chifflot8, x='size', y='time', line_color='mediumseagreen')

p1.legend.location = "top_left"

p2.square(source=source_cublasT_errors_chifflot8, x='sizes', y='cuBLAS_Tensor', legend_label='cuBLAS_Tensor', color='tomato', fill_alpha=0.4)
#p2.line(source=source_cublasT_errors, x='sizes', y='cuBLAS_Tensor', line_color='tomato')
p2.square(source=source_cublas_errors_chifflot8, x='sizes', y='cuBLAS', legend_label='cuBLAS', color='mediumseagreen', fill_alpha=0.4)
#p2.line(source=source_cublas_errors, x='sizes', y='cuBLAS', line_color='mediumseagreen')
p2.y_range.end = maxi*1.1
p2.legend.location = "top_left"

#show(gridplot([p1, p2], ncols=2))
show(column(Div(text="Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU - Nvidia Tesla V100-PCIE-32GB (32 GiB)"), row(p1, p2)))

In [ ]:
# SyCL
i = 0

p1 = figure(title="Time", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

for label in labels_chifflot8_sycl_gpu:
    times = list()
    errors = list()
    data_err = data_chifflot8_sycl_gpu[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors = list(map(float, errors))
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    
    source_sycl_gpu_chifflot8 = ColumnDataSource(data_plot)

    
    p1.square(source=source_sycl_gpu_chifflot8, x='size', y='time', legend_label=labels_chifflot8_sycl_gpu[i], fill_color=None, line_color='mediumpurple')
    p1.line(source=source_sycl_gpu_chifflot8, x='size', y='time', line_color='mediumpurple')
    p1.segment(source=source_sycl_gpu_chifflot8, x0='size', y0='error_low', x1='size', y1='error_high', line_width=2)
    
    p1.square(source=source_cublasT_chifflot8, x='size', y='time', legend_label='cuBLAS_Tensor', fill_color=None, line_color='tomato')
    p1.line(source=source_cublasT_chifflot8, x='size', y='time', line_color='tomato')
    p1.segment(source=source_cublasT_chifflot8, x0='size', y0='error_low', x1='size', y1='error_high', line_width=2)
    
    p1.square(source=source_cublas_chifflot8, x='size', y='time', legend_label='cuBLAS', fill_color=None, line_color='mediumseagreen')
    p1.line(source=source_cublas_chifflot8, x='size', y='time', line_color='mediumseagreen')
    p1.segment(source=source_cublas_chifflot8, x0='size', y0='error_low', x1='size', y1='error_high', line_width=2)

    i=i+1
    
p1.legend.location = "top_left"  

show(column(Div(text="Intel Xeon Gold 6126 (Skylake), x86_64, 2.60GHz, 2 CPUs/node, 12 cores/CPU - Nvidia Tesla V100-PCIE-32GB (32 GiB)"), row(p1)))

--------------------------

# Intel Cloud - 4th Generation Intel® Xeon®

## Data sources

In [2]:
data_intel_cpu = json.load(open('intelCloud/results_cpu.json', 'r'))
labels_intel_cpu = [key for key, value in data_intel_cpu.items()]

data_intel_sycl_cpu = json.load(open('intelCloud/results_sycl_cpu.json', 'r'))
labels_intel_sycl_cpu = [key for key, value in data_intel_sycl_cpu.items()]

## Performace CPU

In [3]:
# 4th Generation Intel® Xeon® Scalable processors (8480)
# 112 cores, 2 sockets, 512 GB memory, 2 TB disk

colors = ['green', 'red', 'blue', 'darkturquoise', 'coral']
i = 0

p1 = figure(title="Time", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'
p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

errors_total = list()

for label in labels_intel_cpu:
    times = list()
    errors = list()
    data_err = data_intel_cpu[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))
    
    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    
    errors_total += [errors]
    errors = list(map(float, errors))
    errors_t = dict()
    errors_t['sizes'] = sizes
    for key, value in zip(labels_intel_cpu, errors_total):
        errors_t[key] = value
        
    source_intel_cpu_errors = ColumnDataSource(pd.DataFrame(errors_t))
    
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending=True)
    data_plot = data_plot.reset_index(drop=True)
    
    source_intel_cpu = ColumnDataSource(data_plot)
    
    p1.square(source=source_intel_cpu, x='size', y='time', legend_label=labels_intel_cpu[i], fill_color=None, line_color=colors[i])
    p1.line(source=source_intel_cpu, x='size', y='time', line_color=colors[i])
    
    if label != 'serial':
        #p2.line(source=source_cpu_errors, x='sizes', y=label, legend_label=labels[i], color=colors[i])
        p2.square(source=source_intel_cpu_errors, x='sizes', y=label, legend_label=labels_intel_cpu[i], color=colors[i])
    
    if label == 'mkl':
        data_mkl_intel = data_plot.copy(deep=True)
        error_mkl_intel = pd.DataFrame(errors_t)
        
    if label == 'openMP':
        data_openMP_intel = data_plot.copy(deep=True)
        error_openMP_intel = pd.DataFrame(errors_t)
        
    i=i+1

maxi = max(max([sublist[:] for sublist in errors_total]))

p2.y_range.end = maxi*1.1
p1.legend.location = "top_left"
p2.legend.location = "top_left"

#show(gridplot([p1, p2], ncols=2))
show(column(Div(text="4th Generation Intel® Xeon® Scalable processors (8480), 112 cores, 2 sockets, 512 GB memory"), row(p1, p2)))

In [4]:
# 4th Generation Intel® Xeon® Scalable processors (8480), 112 cores, 2 sockets, 512 GB memory
p1 = figure(title="Execution", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")
p2 = figure(title="Error", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'

p2.xaxis.axis_label = 'Matrix size (N x N)'
p2.yaxis.axis_label = 'MSE'

source_mkl_intel = ColumnDataSource(data_mkl_intel)
source_mkl_errors_intel = ColumnDataSource(error_mkl_intel)
source_openmp_intel = ColumnDataSource(data_openMP_intel)
source_openmp_errors_intel = ColumnDataSource(error_openMP_intel)

p1.square(source=source_mkl_intel, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
p1.line(source=source_mkl_intel, x='size', y='time', line_color='blue')
p1.square(source=source_openmp_intel, x='size', y='time', legend_label='openMP', fill_color=None, line_color='darkturquoise')
p1.line(source=source_openmp_intel, x='size', y='time', line_color='darkturquoise')

p1.legend.location = "top_left"

p2.square(source=source_mkl_errors_intel, x='sizes', y='mkl', legend_label='mkl', color='blue', fill_alpha=0.4)
#p2.line(source=source_mkl_errors, x='sizes', y='mkl', line_color='blue')
p2.square(source=source_openmp_errors_intel, x='sizes', y='openMP', legend_label='openMP', color='darkturquoise', fill_alpha=0.4)
#p2.line(source=source_openmp_errors, x='sizes', y='openMP', line_color='darkturquoise')
p2.y_range.end = maxi*1.1
p2.legend.location = "top_left"

#show(gridplot([p1, p2], ncols=2))
show(column(Div(text="4th Generation Intel® Xeon® Scalable processors (8480), 112 cores, 2 sockets, 512 GB memory"), row(p1, p2)))

In [6]:
# SyCL 4th Generation Intel® Xeon® Scalable processors (8480), 112 cores, 2 sockets, 512 GB memory

i = 0

colors = ['green', 'red', 'blue', 'black', 'coral']
p1 = figure(title="Time", tools=TOOLS, width=450, height=450, background_fill_color="#fafafa")

p1.xaxis.axis_label = 'Matrix size (N x N)'
p1.yaxis.axis_label = 'Time ( ms )'


for label in labels_intel_sycl_cpu:
    times = list()
    errors = list()
    data_err = data_intel_sycl_cpu[label]
    sizes = [key for key, value in data_err.items()]
    for size in sizes:
        times.append(data_err[size]['time'])
        errors.append(data_err[size]['error'])
    sizes = list(map(int, sizes))

    data_plot = pd.DataFrame({'size':sizes, 'time':times, 'error':errors})
    # Add error bars to the DataFrame
    errors = list(map(float, errors))
    data_plot['error_low'] = data_plot['time'] - np.array(errors)
    data_plot['error_high'] = data_plot['time'] + np.array(errors)
    data_plot = data_plot.sort_values(by=['size'], ascending = True)
    data_plot = data_plot.reset_index(drop=True)
    
    source_sycl_intel = ColumnDataSource(data_plot)
    
    p1.square(source=source_sycl_intel, x='size', y='time', legend_label=labels_intel_sycl_cpu[i], fill_color=None, line_color='darkturquoise')
    p1.line(source=source_sycl_intel, x='size', y='time', line_color='darkturquoise')
    
    p1.square(source=source_mkl_intel, x='size', y='time', legend_label='mkl', fill_color=None, line_color='blue')
    p1.line(source=source_mkl_intel, x='size', y='time', line_color='blue')

    p1.square(source=source_openmp_intel, x='size', y='time', legend_label='openMP', fill_color=None, line_color='darkturquoise')
    p1.line(source=source_openmp_intel, x='size', y='time', line_color='darkturquoise')
    
    i=i+1

show(column(Div(text="4th Generation Intel® Xeon® Scalable processors (8480), 112 cores, 2 sockets, 512 GB memory"), row(p1)))